In [ ]:
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
import scipy
import os

def create_spectrogram(audio_path, title, save_path):
    try:
        y, sr = librosa.load(audio_path, sr=None)
        n_fft = 2048
        hop_length = 512
        D = librosa.stft(y, n_fft=n_fft, hop_length=hop_length, window='hann')
        S_db = librosa.amplitude_to_db(np.abs(D), ref=np.max)

        plt.figure(figsize=(12, 6))
        librosa.display.specshow(S_db,sr=sr,hop_length=hop_length,x_axis='time',y_axis='log')
        plt.colorbar(format='%+2.0f dB')
        plt.title(title)
        plt.xlabel("Время (с)")
        plt.ylabel("Частота (Гц)")
        plt.tight_layout()
        plt.savefig(save_path)
        plt.close()

        print(f"Спектрограмма сохранена в {save_path}")
        return y, sr
    except Exception as e:
        print(f"Ошибка при обработке файла {audio_path}: {e}")
        return None, None


audio_files = {
    "a.wav": ("Гласный 'А'", "a_spectrogram.png"),
    "i.wav": ("Гласный 'И'", "i_spectrogram.png"),
    "bark.wav": ("Лай собаки", "bark_spectrogram.png"),
}

audio_data = {}

for audio_path, (title, save_path) in audio_files.items():
    if os.path.exists(audio_path):
        y, sr = plot_spectrogram(audio_path, title, save_path)
        if y is not None and sr is not None:
            audio_data[audio_path] = (y, sr)
    else:
        print(f"Файл {audio_path} не найден!")


def find_min_max_frequency(y, sr):
    try:
        yf = np.fft.fft(y)
        T = 1.0 / sr
        xf = np.linspace(0.0, 1.0/(2.0*T), len(y)//2)

        indices = np.where(np.abs(yf[:len(y)//2]) > 0.003*np.max(np.abs(yf[:len(y)//2])))[0]

        if len(indices) > 0:
            min_freq = xf[indices[0]]
            max_freq = xf[indices[-1]]
            return min_freq, max_freq
        else:
            return None, None
    except Exception as e:
        print(f"Ошибка при нахождении мин/макс частоты: {e}")
        return None, None


for audio_path, (y, sr) in audio_data.items():
    min_freq, max_freq = find_min_max_frequency(y, sr)
    if min_freq is not None and max_freq is not None:
        print(f"Минимальная частота для {audio_path}: {min_freq:.2f} Гц")
        print(f"Максимальная частота для {audio_path}: {max_freq:.2f} Гц")
    else:
        print(f"Не удалось определить мин/макс частоту для {audio_path}")


def find_dominant_frequency(y, sr):
    try:
        acf = librosa.autocorrelate(y, max_size=len(y))

        peak_idx = scipy.signal.find_peaks(acf[:len(acf)//2])[0]
        if len(peak_idx) > 0:
            fundamental_frequency = sr / peak_idx[0]
            return fundamental_frequency
        else:
            return None
    except Exception as e:
        print(f"Ошибка при нахождении основного тона: {e}")
        return None

for audio_path, (y, sr) in audio_data.items():
    fundamental_freq = find_dominant_fundamental_frequency(y, sr)
    if fundamental_freq is not None:
        print(f"Наиболее тембрально окрашенный основной тон для {audio_path}: {fundamental_freq:.2f} Гц")
    else:
        print(f"Не удалось определить основной тон для {audio_path}")

def find_formants(y, sr, audio_path):
    try:
        window_duration = 0.05
        time_step = 0.1
        frequency_step = 50

        window_size = int(window_duration * sr)
        hop_length = int(time_step * sr)
        n_fft = 4096  
        formant_candidates = []

        for i in range(0, len(y) - window_size, hop_length):
            window = y[i:i + window_size]
            window = window * np.hamming(len(window))

            yf = np.fft.fft(window, n=n_fft)
            xf = np.fft.fftfreq(n_fft, d=1/sr)

            positive_frequencies = xf[:n_fft//2]
            magnitude_spectrum = np.abs(yf[:n_fft//2])
            freq_resolution = sr / n_fft
            min_peak_distance = int(frequency_step / freq_resolution)
            min_peak_distance = max(1, min_peak_distance)

            peak_indices = scipy.signal.find_peaks(
                magnitude_spectrum,
                distance=min_peak_distance,
                prominence=0.1 * np.max(magnitude_spectrum)
            )[0]

            peaks_freq = positive_frequencies[peak_indices]
            peaks_magnitude = magnitude_spectrum[peak_indices]

            for freq, mag in zip(peaks_freq, peaks_magnitude):
                if freq > 50:
                    formant_candidates.append((freq, mag))

        formant_candidates.sort(key=lambda x: x[1], reverse=True)

        top_formants = []
        seen_frequencies = set()

        for freq, mag in formant_candidates:
            if all(abs(freq - f) > 100 for f in seen_frequencies):
                top_formants.append((freq, mag))
                seen_frequencies.add(freq)
                if len(top_formants) >= 3:
                    break

        if len(top_formants) > 0:
            print(f"\nТоп-3 форманты для {audio_path}:")
            for i, (freq, mag) in enumerate(top_formants, 1):
                print(f"  Форманта {i}: {freq:.2f} Гц, Амплитуда: {mag:.2f}")
        else:
            print(f"\nНе удалось определить форманты для {audio_path}")

    except Exception as e:
        print(f"\nОшибка при нахождении формант для {audio_path}: {e}")

print("\Анализ формант:")
for audio_path, (y, sr) in audio_data.items():
    find_formants(y, sr, audio_path)

Спектрограмма сохранена в a_spectrogram.png
Спектрограмма сохранена в i_spectrogram.png
Спектрограмма сохранена в bark_spectrogram.png
Минимальная частота для a.wav: 20.27 Гц
Максимальная частота для a.wav: 5571.86 Гц
Минимальная частота для i.wav: 0.00 Гц
Максимальная частота для i.wav: 7610.02 Гц
Минимальная частота для bark.wav: 0.00 Гц
Максимальная частота для bark.wav: 8399.46 Гц
Наиболее тембрально окрашенный основной тон для a.wav: 125.33 Гц
Наиболее тембрально окрашенный основной тон для i.wav: 301.89 Гц
Наиболее тембрально окрашенный основной тон для bark.wav: 350.36 Гц

Анализ формант:

Топ-3 форманты для a.wav:
  Форманта 1: 105.47 Гц, Амплитуда: 50.44
  Форманта 2: 515.62 Гц, Амплитуда: 20.65
  Форманта 3: 621.09 Гц, Амплитуда: 7.17

Топ-3 форманты для i.wav:
  Форманта 1: 105.47 Гц, Амплитуда: 27.97
  Форманта 2: 281.25 Гц, Амплитуда: 23.96
  Форманта 3: 398.44 Гц, Амплитуда: 2.78

Топ-3 форманты для bark.wav:
  Форманта 1: 105.47 Гц, Амплитуда: 45.87
  Форманта 2: 292.97 